# Advanced Lane Lines Detection Pipeline

## Calibrating the Camera

In [49]:
# globals used internally by the calibrateCamera function
gCamCalMatrix = None
gCamCalDistortionCoeff = None 

def calibrateCamera():
    # return if the global camCalMatrix already exists
    global gCamCalMatrix
    global gCamCalDistortionCoeff
    
    if(gCamCalMatrix is not None):
        return gCamCalMatrix, gCamCalDistortionCoeff
    
    return gCamCalMatrix, gCamCalDistortionCoeff

## Correcting Image Distortion

In [50]:
def distortionCorrection(img, matrix, coeff):
    correctedImage = img
    
    return correctedImage

## Accentuating Lane Lines

In [51]:
def colorTransform(img):
    # TODO: pass in type
    return img

In [52]:
def sobelGradientBinary(img):
    # TODO: pass in parameters for threshes
    return img

In [53]:
def whiteAndYellowBinary(img):
    # TODO: highlight the white and yellow parts of image
    # TODO: pass in type
    return img

In [54]:
def accentuateLaneLines(img):
    # TODO: put this function in a different file
    colorTransformImage = colorTransform(img)
    sobelGradientBinaryImage = sobelGradientBinary(img)
    whiteAndYellowBinaryImage = whiteAndYellowBinary(colorTransformImage)
    
    # TODO: combine these images together for the final result
    combinedBinary = img
    
    return combinedBinary

## Applying a Bird's Eye View Perspective Transform

In [55]:
def birdsEyeView(img):
    birdsEyeViewImage = img
    
    # TODO: use source and destination points from example writeup
    
    return birdsEyeViewImage

## Identifying Lane Lines from a Bird's Eye View Image

In [56]:
def identifyLaneLines(img):
    # TODO: run function to detect
    # TODO: run function to use previous
    # TODO: move this functions and it's dependent functions
    # to a separate notebook
    binaryPerspectiveImage = img
    
    # TODO: getRadiusOfCurvature function
    radiusOfCurvature = img
    
    return binaryPerspectiveImage, radiusOfCurvature

## Reversing The Bird's Eye View Perspective Transform

In [57]:
def birdsEyeViewToOriginal(img):
    # TODO: add parameters (inversion matrix)
    originalImage = img
    return originalImage

## Adding Detected Lines Overlay to Original Image

In [58]:
def overlayDetectedLines(img, binaryDetectedLines):
    # TODO: add weight parameter
    overlayedImage = img
    
    return img

## Running the Lane Line Detection Pipeline

In [59]:
def pipeline(img):
    # We must calibarate our camera with the provided chessboard images
    # (initializes on first call, uses globals on successive calls)
    camCalMatrix, camCalDistortionCoeff = calibrateCamera()
    
    # Apply distortion correction to the image with the computed
    # camera calibration matrix and distortion coefficients
    distortionCorrectedImage = distortionCorrection(img, camCalMatrix, camCalDistortionCoeff)
    
    # Accentuate possible lane lines via color transform, gradients, and color highlighting (white and yellow)
    binaryImage = accentuateLaneLines(distortionCorrectedImage)
    
    # Apply a perspective transform (birds-eye view) to the binary image
    binaryBirdsEyeViewImage = birdsEyeView(binaryImage)
    
    # Identify left and right lane lines on the perspective transformed binary image
    binaryLinesBirdsEyeViewImage, radiusOfCurvature = identifyLaneLines(binaryBirdsEyeViewImage)
    
    # TODO: positionFromLaneCenter
    
    # Transform the birds eye view image back to the original perspective
    binaryLinesImage = birdsEyeViewToOriginal(binaryLinesBirdsEyeViewImage)
    
    # Overlay the original image with the detected lane lines binary image
    detectedLinesOverlayImage = overlayDetectedLines(img, binaryLinesImage)
    
    return detectedLinesOverlayImage
    
    

## Applying Pipeline To a Video

In [71]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import display, HTML

def pipelineVideo(inputPath, outputPath, subclip=False): 
    videoClip = None
    
    # apply subclip if subclip flag is set
    if subclip:
        videoClip = VideoFileClip(inputPath).subclip(0,5)
    else:
        videoClip = VideoFileClip(inputPath)
    
    # apply pipeline to video
    processedClip = videoClip.fl_image(pipeline)
    
    # write clip to the specified output path
    %time processedClip.write_videofile(outputPath, audio=False)

## Pipelining and Displaying Video

In [75]:
def pipelineAndDisplayVideo(inputPath, outputPath, subclip=False):
    pipelineVideo(inputPath, outputPath, subclip)
    
    display(HTML("""
    <video width="100%" height="100%" controls>
      <source src="{0}">
    </video>
    """.format(outputPath)))

## Running Pipeline on Project Video

In [76]:
pipelineAndDisplayVideo("project_video.mp4", "output_videos/project_video_output.mp4", True)

[MoviePy] >>>> Building video output_videos/project_video_output.mp4
[MoviePy] Writing video output_videos/project_video_output.mp4


 99%|█████████▉| 125/126 [00:23<00:00,  5.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video_output.mp4 

Wall time: 31.8 s


## Running Pipeline on Challenge Video

In [66]:
pipelineVideo("challenge_video.mp4", "output_videos/challenge_video_output.mp4", True)

[MoviePy] >>>> Building video output_videos/challenge_video_output.mp4
[MoviePy] Writing video output_videos/challenge_video_output.mp4


100%|██████████| 150/150 [00:41<00:00,  3.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video_output.mp4 

Wall time: 53.3 s


## Running Pipeline on Harder Challenge Video

In [ ]:
pipelineVideo("harder_challenge_video.mp4", "output_videos/harder_challenge_video_output.mp4", True)